<a href="https://colab.research.google.com/github/mdallas1/shared_code/blob/main/L5_5_iterative_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install octave

# Iterative Methods

In a letter Gauss wrote to his student around 1823, he said the following.

> Almost every evening I make a new edition of the tableau, where there's always a little touch-up to do. With the monotony of measurement work, this always makes for a pleasant diversion; one can immediately see whether something questionable has crept in, which is still desirable, etc.
I recommend this method for imitation. You will hardly ever directly eliminate again—at least not when you have more than 2 unknowns.

The "tableau" he's referring is a list of approximate solution solutions to a linear system. The new ones generated from the old ones. This describes not a direct method for solving a linear system, but an iterative one (or indirect as Gauss would put it). Gauss was clearly a fan, and we'll see in the following lectures what makes iterative methods attractive.

## Introduction

So far we've studied *direct methods* for solving linear systems $A\mathbf{x}=\mathbf{b}$. Direct methods will compute the exact solution in a finite (finite does not mean few) number of steps. This is great in some situations, but not always. Direct methods may be too expensive or too memory intensive when working with very large problems.

An alternate approach is to use an *iterative method*. Like in chapter 2, when we were solving nonlinear equations, an iterative method for computing the solution $\mathbf{x}^*$ to the linear system $A\mathbf{x}=\mathbf{b}$ starts an initial guess $\mathbf{x}^{(0)}$, and generates a sequence $\{\mathbf{x}^{(k)}\}$ such that $\mathbf{x}^{(k)}\to \mathbf{x}^*$ as the $k\to\infty$.

## Constructing an iterative method

We'll first discuss the simplest general approach to developing a fixed point method. This discussion isn't in our book, but we will join back up with the book soon.

One way to construct an iterative method for solving $A\mathbf{x} = \mathbf{b}$ is to transform it into a *fixed point problem*. Recall that the problems of finding the root of a function is essentially equivalent to finding the fixed point of a related function. Precisely, $x^*$ is a solution to $f(x) = 0$ if and only if $x^*$ is a fixed point of $g(x) = x+f(x)$.

Considering the problem at hand, observe that $\mathbf{x}^*$ is a solution to $A\mathbf{x}=\mathbf{b}$ if and only if $\mathbf{x}^*$ is a root of the residual function $r(\mathbf{x}) = \mathbf{b}-A\mathbf{x}$. Further,

$$ r(\mathbf{x}^*) = 0 \iff \mathbf{x}^* = (I-A)\mathbf{x}^*+b.$$

Thus, solving $A\mathbf{x}=b$ is equivalent to finding a fixed point of $g(\mathbf{x}) = (I-A)\mathbf{x} + \mathbf{b}$. We won't focus on this particular fixed point function. Rather, we will focus on a more general form that allows for a *preconditioner*.

We start by *splitting* the matrix $A$.  

$$ A = P - (P-A).$$

Here $P$ is a nonsingular matrix called the *preconditioner* of $A$. The idea of a preconditioner is to somehow make the system "better" before we try and solve it. We've seen this idea once already when we discussed LU with partial pivoting. If $A\mathbf{x}=\mathbf{b}$ is "bad" in the sense that $A$ doens't have an LU decomposition, then, as long as $A$ is nonsingular, there's a permutation matrix $P$ (which is nonsingular) such that $PA$ does have an LU factorization. So we first multiply by $P$ to get $PA\mathbf{x} = P\mathbf{b}$, and we then solve this new "better" system.  

Now noting that

$$ 0 = \mathbf{b} - A\mathbf{x} \iff P\mathbf{x} = (P-A)\mathbf{x} + \mathbf{b}.$$

Since $P^{-1}$ exists, our problem is reduced to computing a fixed point value of $g(\mathbf{x}) = P^{-1}(P-A)\mathbf{x}+P^{-1}\mathbf{b}$. By setting $B = P^{-1}(P-A)$ and $\mathbf{g} = P^{-1}\mathbf{b}$, we find that this fixed point iteration is of the general form

$$ \mathbf{x}^{(k+1)} = B\mathbf{x}^{(k)} + \mathbf{g}.$$

What can we say theoretically about the convergence of this very general iterative scheme? Ostrowski's theorem still applies, but here it would say that if the matrix $B$ is less than one in some sense, then $\mathbf{x}^{(k+1)} \to \mathbf{x}^*$ at least locally. The question now is what do we mean for a matrix $B$ to be less than one?

First, note that since $\mathbf{x}^* = B\mathbf{x}^* + \mathbf{b}$, if we subtract this equation from $\mathbf{x}^{(k+1)} = B\mathbf{x}^{(k)} + \mathbf{g}$, and define $\mathbf{e}^{(k)} = \mathbf{x}^{(k)}-\mathbf{x}^*$, then

$$ \mathbf{e}^{(k+1)} = B\mathbf{e}^{(k)}.$$

In our analysis of the accuracy of a computed solution, we learned that if a matrix $A$ is SPD (symmetric and positive definite), then for any vector $\mathbf{v}$, $\|A\mathbf{v}\| \leq \lambda_{max}(A)\|\mathbf{v}\|$. Here, $\|v\| = \sqrt{v_1^2+\cdots+v_n^2}$ is the Euclidean norm of the vector $\mathbf{v}$. If we assume that our matrix $B$ here is SPD, then we have that

$$ \|e^{(k+1)}\| = \|B\mathbf{e}^{(k)}\| \leq \lambda_{max}(B)\|\mathbf{e}^{(k)}\|,$$

where $\lambda_{max}(B)$ is the largest eigenvalue of $B$. The largest eigenvalue (in magnitude) of matrix is known as the *spectral radius*, and is denoted $\rho(B)$. So we can write the above inequality as

$$ \|e^{(k+1)}\| \leq \rho(B)\|\mathbf{e}^{(k)}\|.$$


This inequality holds for any $k \geq 0$, so we can apply it to $\mathbf{e}^{(k)}$, $\mathbf{e}^{(k-1)}$, and so on until we reach $\mathbf{e}^{(0)}$ and obtain

$$ \|e^{(k+1)}\| \leq \rho(B)^{k+1}\|\mathbf{e}^{(0)}\|.$$

Thus, if $\rho(B) < 1$, it follows that $\|\mathbf{e}^{(k+1)}\to 0$ as $k\to\infty$. In other words, the fixed point iteration

$$ \mathbf{x}^{(k+1)} = B\mathbf{x}^{(k)} + \mathbf{g}$$

 converges to the fixed point $\mathbf{x}^*$. It turns out that this kind of result holds even if $B$ is not SPD.



> **Proposition 5.2** If $\mathbf{x}^* = B\mathbf{x}^*+\mathbf{g}$, then $\mathbf{x}^{(k+1)} = B\mathbf{x}^{(k)} + \mathbf{g}$ converges to $\mathbf{x}^*$ if and only if $\rho(A) < 1$.


The smaller the spectral radius $\rho(B)$, the fewer iterations are required to reach the desired tolerance.

It is in this sense that Ostrowski's theorem applies. When we vaguely said that $B$ must be less than one, we mean precisely that the spectral radius, the largest eigenvalue in magnitude, must be less than one.


## Another form of the fixed point iteration

Suppose we have the preconditioned fixed point scheme

$$ P\mathbf{x}^{(k+1)} = (P-A)\mathbf{x}^{(k)} + \mathbf{b}.$$

If we distribute $\mathbf{x}^{(k)}$ on the right-hand-side, we find that this iteration is equivalent to

$$ P\mathbf{x}^{(k+1)} = P\mathbf{x}^{(k)} + \mathbf{r}^{(k)},$$

where $\mathbf{r}^{(k)} := \mathbf{b}-A\mathbf{x}^{(k)}$ is the *residual* as step $k$. One may also want to include a parameter $\alpha_k$ (which is a scalar) that can improve the convergence properties. Including such a parameter leads to an iteration of the form

$$ P\mathbf{x}^{(k+1)} = P\mathbf{x}^{(k)} + \alpha_k\mathbf{r}^{(k)}.$$

Thinking algorithmically, if we have $\mathbf{x}^{(k)}$, then to obtain the next iterate $\mathbf{x}^{(k+1)}$, we perform the following.

1. Solve the system $P\mathbf{z}^{(k)} = \mathbf{r}^{(k)}$.
2. Set $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_k\mathbf{z}^{(k)}$.

The preconditioning matrix $P$ should be easy to invert, since we're solving the system $P\mathbf{z}^{(k)} = \mathbf{r}^{(k)}$ at each step $k$. For example, if $P$ is diagonal, or tridiagonal, then $P\mathbf{z}^{(k)} = \mathbf{r}^{(k)}$ is very cheap to solve. In the next sections, we'll study two specific iterative techniques known as the Jacobi method and the Gauss-Seidel method.

## The Jacobi Method

To apply the Jacobi iteration, we require that the diagonal entries of $A$ be nonzero. The reason why will be clear soon. The Jacobi method, or Jacobi iteration is obtained by setting $\alpha_{k}=1$ for all $k$, and selecting $P = D =  diag(a_{11},a_{22},...,a_{nn})$ as our preconditioning matrix. Thus,

$$ P\mathbf{x}^{(k+1)} = (P-A)\mathbf{x}^{(k)} + \mathbf{b}$$

becomes

$$ D\mathbf{x}^{(k+1)} = (D-A)\mathbf{x}^{(k)} + \mathbf{b} = \mathbf{b} - (A-D)\mathbf{x}^{(k)}.$$

Since the diagonal entries of $A$ are nonzero, $D$ is invertible with $D^{-1} = diag(1/a_{11},...,1/a_{nn})$, and solving for $\mathbf{x}^{(k+1)}$ yields the Jacobi method.

$$
\mathbf{x}^{(k+1)} = D^{-1}(\mathbf{b} - (A-D)\mathbf{x}^{(k)}),\hspace{1em} k\geq 0.
$$

Componentwise, the Jacobi method takes the form

$$
x_i^{(k+1)} = \dfrac{1}{a_{ii}} \left( b_i - \sum_{j=1,j\neq i}^n a_{ij}x_j^{(k)}\right),\hspace{1em} i=1,...,n.
$$

Since this is an iterative method, we have to select $\mathbf{x}^{(0)}$ ourselves. The good news is that, as long as the spectral radius of our iteration matrix is less than one, we have guaranteed convergence from any $\mathbf{x}^{(0)}$. In the Jacobi method, our iteration matrix is

$$ B = D^{-1}(D-A) = I - D^{-1}A. $$

Observe that the better of an approximation $D^{-1}$ is to $A^{-1}$, the closer $B$ is to the zero matrix, and thus the smaller the spectral radius. Recall that $D$ was our preconditioning matrix. Thus, the "more digaonal" $A$ is, the better of a preconditioner $D$ is. This is the intuition behind the following proposition. The conditions of which are easier to check than those of Proposition 5.2 above.

> **Proposition 5.3** If the $n\times n$ matrix $A$ is strictly diagonally dominant by row, then the Jacobi method converges.





In [ ]:
#@title Jacobi method
%%writefile jacobi_method.m

function out = jacobi_method(A,b,x)
  % ==================================
  % SOLVES Ax = b WITH INITIAL GUESS x
  % WITH JACOBI.
  % ==================================

  n = length(A); tol = 1e-12; maxiters = 300;
  iters = 0; err = tol+1;

  % UPDATE X WITH JACOBI ITERATION
  while err > tol && iters < maxiters
    D = diag(diag(A));
    xold = x;
    for i = 1:n
      x(i) = (b(i) - (A(i,1:end)-D(i,1:end))*xold)/A(i,i);
    end

  % THE RELATIVE RESIDUAL (BACKWARD ERROR) IS OUR ERROR ESTIMATOR
  err = norm(b - A*x)/norm(b);

  % INCREMENT ITERS
  iters += 1;

  % UNCOMMENT TO GET PRINTOUT EACH ITERATION
  % fprintf("Iter\t| Err\n=================\n %g\t| %.1e\n=================\n",iters,err)
  end

  fprintf("Terminal residual: %.2e. Iterations: %g\n",err,iters)
  out = x;
end


In [ ]:
#@title Jacobi method example
%%writefile jacobi_example.m

% DEFINE MATRIX. NOTE STRICT DIAGONAL DOMINANCE BY ROW.
A = [10 2 -1 4 ; 1 7 1 -3 ; 1 0 11 4 ; 1 2 6 13];

% CHOOSE RIGHT HAND SIDE SO THAT EXACT SOLUTION IS X = 1
x_soln = ones(4,1);
b = A*x_soln;

x = jacobi_method(A,b,rand(4,1));

disp("")

fprintf("True error: %.2e\n",norm(x-x_soln)/norm(x_soln));

% COMPUTE THE EIGENVALUES OF B = D^{-1}(D-A).
% DOES THIS EXPLAIN THE OBSERVED BEHAVIOR?
D = diag(diag(A));
B = inv(D)*(D-A);
abs(eig(B))

In [ ]:
!octave -W jacobi_example.m

In [ ]:
#@title Non-diagonally dominant example
%%writefile non_diag_dom.m

A = [1 2 -1 4 ; 1 1 2 -3 ; 1 0.5 eps 4 ; 1 2 6 0.5];

x_soln = ones(4,1);
b = A*x_soln;

x = jacobi_method(A,b,rand(4,1));

% COMPUTE THE EIGENVALUES OF B = D^{-1}(D-A).
% DOES THIS EXPLAIN THE OBSERVED BEHAVIOR?
D = diag(diag(A));
B = inv(D)*(D-A);
abs(eig(B))

In [ ]:
!octave -W non_diag_dom.m

The code cells above demonstrate what happens when $A$ is strictly diagonally dominant by row, and what happens when it is not. The key is that if the former holds, then we can show that $\rho(B) < 1$ where $B = D^{-1}(D-A)$.

Note that the Jacobi method is highly parallelizable since we can update each component of $\mathbf{x}^{(k)}$ independent of the others. This is not the case with the next method.


## The Gauss-Seidel Method

The idea behind Gauss-Seidel is that, in the Jacobi method, when we're computing $x_i^{(k+1)}$, we already have $x_j^{(k+1)}$ for $j=1,...,i-1$. So instead of just using $x_j^{(k)}$ when computing $x_i^{(k+1)}$, let's use $x_j^{(k+1)}$. Perhaps this will lead to a better iterative scheme. In the Jacobi method, we compute $x_i^{(k+1)}$ by

$$ x_i^{(k+1)} = \dfrac{1}{a_{ii}}\left(b_i - \sum_{j=1,j\neq i} a_{ij}x_j^{(k)} \right)$$

To obtain the Gauss-Seidel method, for each $j < i$, replace $x_j^{(k)}$ with $x_j^{(k+1)}$:

$$ x_i^{(k+1)} = \dfrac{1}{a_{ii}}\left(b_i -\sum_{j=1}^{i-1} a_{ij}x_j^{(k+1)}- \sum_{j=i+1}^n a_{ij}x_j^{(k)}\right)$$


How does Gauss-Seidel fit in the general framework

$$ P\mathbf{x}^{(k+1)} = P\mathbf{x}^{(k)} +\alpha_k\mathbf{r}_k ?$$

It turns out that if we take $\alpha_k=1$ for all $k$, and set $P = D-E$, where $E$ is the lower triangular matrix with entries $-a_{ij}$ for $2\leq i\leq n$ and $1\leq j \leq i-1$. To see this, note that we can write the entrywise Gauss-Seidel update as

$$ D\mathbf{x}^{(k+1)} = \mathbf{b} + E\mathbf{x}^{(k+1)} - A\mathbf{x}^{(k)} + E\mathbf{x}^{(k)} - D\mathbf{x}^{(k)}.$$

A bit of rearranging yields

$$ (D-E)\mathbf{x}^{(k+1)} = \mathbf{b} - (A-E+D)\mathbf{x}^{(k)} = (D-E)\mathbf{x}^{(k)} + \mathbf{r}^{(k)}.$$

Neither the Jacobi method nor the Gauss-Seidel method is better in general. For instance, as we already noted, the Jacobi method is highly parallelizable, while Gauss-Seidel is not. There are, however, some special cases there is a clear winner.

> **Proposition 5.4** Let $A$ be an $n\times n$ tridiagonal matrix whose diagonal elements are all non-zero. Then
1. Jacobi and Gauss-Seidel both either converge or diverge.
2. In the case of convergence, Gauss-Seidel converges faster than Jacobi. More precisely, the spectral radius of Gauss-Seidel iteration matrix is the square of the Jacobi iteratiox matrix.


In [ ]:
#@title Gauss-Seidel
%%writefile gauss_seidel.m

function out = gauss_seidel(A,b,x)
  % ==================================
  % SOLVES Ax = b WITH INITIAL GUESS x
  % USING GAUSS-SEIDEL.
  % ==================================

  n = length(A); tol = 1e-12; maxiters = 300;
  iters = 0; err = tol+1;

  % UPDATE X WITH GAUSS-SEIDEL ITERATION
  while err > tol && iters < maxiters
    D = diag(diag(A));

    % NOTICE THAT THE ONLY DIFFERENCE HERE FROM JACOBI IS THAT WE USE THE
    % UPDATED X IN EACH STEP.
    for i = 1:n
      x(i) = (b(i) - (A(i,1:end)-D(i,1:end))*x)/A(i,i);
    end

  % THE RELATIVE RESIDUAL (BACKWARD ERROR) IS OUR ERROR ESTIMATOR
  err = norm(b - A*x)/norm(b);

  % INCREMENT ITERS
  iters += 1;

  % UNCOMMENT TO GET PRINTOUT EACH ITERATION
  % fprintf("Iter\t| Err\n=================\n %g\t| %.1e\n=================\n",iters,err)
  end

  fprintf("Terminal residual: %.2e. Iterations: %g\n",err,iters)
  out = x;
end

In [ ]:
#@title Compare Jacobi and Gauss-Seidel
%%writefile ex5_13.m

% EXAMPLE 5.13 ON PAGE 173 OF OUR BOOK

A = zeros(10,10);
A = diag(3*ones(10,1)) + diag(-2*ones(9,1),-1) + diag(-ones(9,1),1)

% CHOOSE RIGHT HAND SIDE SO THAT EXACT SOLUTION IS X = 1
x_soln = ones(10,1);
b = A*x_soln;

% APPLY JACOBI
x_jac = jacobi_method(A,b,zeros(10,1));

% APPLY GAUSS-SEIDEL
x_gs = gauss_seidel(A,b,zeros(10,1));


In [ ]:
!octave -W ex5_13.m